# AI Community @ Семинар № 9, весна 2018
## Вариационный автокодировщик (Variational Autoencoder)

Если обычный автокодировщик конвертирует объект в некоторый код, то _вариационный_ автокодировщик конвертирует его в параметры некоторого распределения.  
Это упрощает генерацию новых сэмплов.

<img src='images/vae.png' width=500>


В дальнейшем будем использовать несколько предположений.  
Пусть $X = \{ x^{(i)} \}_{i=1}^N$ — выборка, полученная из некоторого случайного процесса с параметром $z$. Другими словами, будем считать что она просэмплирована из следующего распределения:
$$ x \sim p_{\theta}(x) = \int p_{\theta}(x | z) p(z) dz,$$
$p(z)$ — априорное распределение на скрытые параметры $z$, $p_{\theta}(x | z)$ — условного распределение элементов выборки при наблюдении скрытого параметра $z$.  
Для упрощения рассуждений будем моделировать условное распределение $p_{\theta}(x | z)$ следующим образом:
$$ p_{\theta}(x | z) \sim \mathcal{N}(f_{\theta}(z), I \sigma^2) $$

Наша задача состоит в том, чтобы найти апостериорное распределение $p_{\theta}(z | x)$, а именно совершить байесовский вывод (variational inference):
$$ p_{\theta}(z|x) = \frac{ p_{\theta}(x|z) p(z)}{p_{\theta}(x)} = \frac{ p_{\theta}(x|z) p(z)}{\int p_{\theta}(x | z) p(z) dz}$$

Проблема: совершить байесовский вывод крайне трудно — для этого требуется посчитать интеграл $p_{\theta}(x)$, а он является intractable — простанство параметра $z$ высокоразмерно и объемно.  
Вместо этого будем приближать истинное апостериорное распределение $p_{\theta}(z|x)$ при помощи распределения $q_{\phi}(z|x)$ с параметром $\phi$.

Для измерения близости распределений принято пользоваться дивергенцией Кульбака-Лейблера:
$$ KL(q(x) || p(x)) = - \int q(x) \log \frac{p(x)}{q(x)} dx = -\mathbb{E}_{x \sim q(x)} \left[ \log \frac{p(x)}{q(x)} \right] = \mathbb{E}_{x \sim q(x)} \left[ \log q(x) - \log p(x) \right]$$
Она не является симметричной.  
<img src='images/kl.png' width=600>  
В первом случае распределение $q$ приближает $p$ в области высоких значений, а во втором — по всему распределению. Далее используется первый случай.

Запишем КЛ-дивергенцию для $q_{\phi}(z|x)$ по отношению к $p_{\theta}(z|x)$ и применим формулу Байеса:
$$ KL(q_{\phi}(z|x) || p_{\theta}(z|x)) = \mathbb{E}_{z \sim q_\phi(z|x)} \left[ \log q_\phi(z|x) - \log p_\theta(x | z) - \log p_\theta(z) \right] + \log p_\theta(x) $$
Заметим, что первое и третье слагаемые под матожиданием можно собрать в еще одну КЛ-дивергенцию:
$$ \mathbb{E}_{z \sim q_\phi(z|x)} \left[ \log q_\phi(z|x) - \log p_\theta(x | z) - \log p_\theta(z) \right] + \log p_\theta(x) = \
KL(q_{\phi}(z|x) || p_{\theta}(z)) - \mathbb{E}_{z \sim q_\phi(z|x)} \left[ \log p_\theta(x|z)\right] + \log p_\theta(x)$$

Получили следующее тождество:
$$ \log p_\theta(x) - KL(q_{\phi}(z|x) || p_{\theta}(z|x)) = \mathbb{E}_{z \sim q_\phi(z|x)} \left[ \log p_\theta(x|z)\right] - KL(q_{\phi}(z|x) || p_{\theta}(z))$$
Слева — то, что мы хотим максимизировать и некоторая неотрицательная ошибка (в виде КЛ-дивергенции). Справа — то, что можем оптимизировать градиентным спуском, а именно матожидание правдоподобия декодера и КЛ-дивергенция между распределением, которое моделирует энкодер и стандартным нормальным распределением. Оптимизируя правую часть — будем оптимизировать левую.

Для простоты будем считать что $p_\theta(z) \sim \mathcal{N}(0, I)$, а $q_\phi(z | x) \sim \mathcal{N}(\mu_\phi(x), \Sigma_\phi(x))$. В таком случае формулы упрощаются:
$$ KL(q_{\phi}(z|x) || p_{\theta}(z)) = \frac{1}{2} (tr(\Sigma_\phi(x)) + \mu_\phi(x)^T \mu_\phi(x) - k - \log \det \Sigma_\phi(x) )$$

### Ссылки:
[0] Статья на Хабре из цикла про автокодировщики. https://habrahabr.ru/post/331552/  
[1] Дивергенция Кульбака-Лейблера, свойства нормального распределения. Материалы курса "Байесовские методы машинного обучения". http://www.machinelearning.ru/wiki/images/d/d0/BMMO11_6.pdf  
[2] Туториал по вариационным автокодировщикам: https://jaan.io/what-is-variational-autoencoder-vae-tutorial/  